In [1]:
import torch
import torchvision
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plta
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torchvision.transforms.functional as fn
from torch import nn
import torch.nn.functional as F
import wandb
import torch.optim as optim
from tqdm import tqdm
import sys
import autoreload
%load_ext autoreload
%autoreload 2
sys.path.append('../scripts/')
from save_model_every_epoch_torch import save_model
import warnings
warnings.filterwarnings("ignore")
from metrics import find_metrics, find_metrics_macro
from save_model_every_epoch_torch import save_model

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
multiple_gpus = True
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        multiple_gpus = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (512, 512))
        img = img / 255.0
        return img, torch.Tensor(self.encoded_labels[idx])

In [8]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [9]:
# incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)

In [10]:
def get_model():
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    return incep_model

In [11]:
categorical_cross_entropy = nn.CrossEntropyLoss()

Without momentum changes

In [12]:
def train_model(train_dataset, val_dataset, epochs, batch_size, load_weights=None):
    model = get_model()
    wandb.init(project='dolphin',
               config = {
                   'arch' : 'InceptionV4',
               })
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    if next(model.parameters()).is_cuda == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss= 0.0
        train_prec, train_rec = 0.0, 0.0
        val_loss = 0.0
        val_prec, val_rec = 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            with tqdm(data_pointers[phase], unit='batch') as tepoch:
                for imgs, labels in tepoch:
                    tepoch.set_description(f'Epoch: {epoch}')
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs = model(imgs)[0]
                        else:
                            outputs = model(imgs)
                        loss = categorical_cross_entropy(outputs, labels)
                        prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    running_loss += loss.item()
                    running_precision += prec.item()
                    running_recall += rec.item()
                    tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_prec = running_precision / len(train_dataset)
                train_rec = running_recall / len(train_dataset)
                print(f'{phase} Loss: {float(train_loss)}')
                print(f'{phase} Precision: {float(train_prec)}')
                print(f'{phase} Recall: {float(train_rec)}')
            else:
                val_loss = running_loss / len(val_dataset)
                val_prec = running_precision / len(val_dataset)
                val_rec = running_recall / len(val_dataset)
                print(f'{phase} Loss: {float(val_loss)}')
                print(f'{phase} Precision: {float(val_prec)}')
                print(f'{phase} Recall: {float(val_rec)}')
        wandb.log({
            'train_loss' : train_loss,
            'val_loss' : val_loss,
            'train_precision' : train_prec,
            'train_recall' : train_rec,
            'val_precision' : val_prec,
            'val_recall' : val_rec
        })

In [13]:
batch_size = 228
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, epochs=50)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0:   6%|▌         | 13/220 [00:54<14:29,  4.20s/batch, loss=3.29, precision=0.0513, recall=0.0224]   


KeyboardInterrupt: 

In [13]:
def train_model(train_dataset, val_dataset, epochs, batch_size, weights_path, load_weights=None):
    model = get_model()
    wandb.init(project='dolphin',
               config = {
                   'arch' : 'InceptionV4',
               })
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.5)
    if next(model.parameters()).is_cuda == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss= 0.0
        train_prec, train_rec = 0.0, 0.0
        val_loss = 0.0
        val_prec, val_rec = 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            with tqdm(data_pointers[phase], unit='batch') as tepoch:
                for imgs, labels in tepoch:
                    tepoch.set_description(f'Epoch: {epoch}')
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs = model(imgs)[0]
                        else:
                            outputs = model(imgs)
                        loss = categorical_cross_entropy(outputs, labels)
                        prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    running_loss += loss.item()
                    running_precision += prec.item()
                    running_recall += rec.item()
                    tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_prec = running_precision / len(train_dataset)
                train_rec = running_recall / len(train_dataset)
                print(f'{phase} Loss: {float(train_loss)}')
                print(f'{phase} Precision: {float(train_prec)}')
                print(f'{phase} Recall: {float(train_rec)}')
            else:
                val_loss = running_loss / len(val_dataset)
                val_prec = running_precision / len(val_dataset)
                val_rec = running_recall / len(val_dataset)
                print(f'{phase} Loss: {float(val_loss)}')
                print(f'{phase} Precision: {float(val_prec)}')
                print(f'{phase} Recall: {float(val_rec)}')
        wandb.log({
            'train_loss' : train_loss,
            'val_loss' : val_loss,
            'train_precision' : train_prec,
            'train_recall' : train_rec,
            'val_precision' : val_prec,
            'val_recall' : val_rec
        })
        save_model(model, epoch, optimizer, multiple_gpus, weights_path)

In [13]:
batch_size = 228
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum/', epochs=50)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 220/220 [04:51<00:00,  1.33s/batch, loss=2.84, precision=0.045, recall=0.0439]   


train Loss: 2.7969882011413576
train Precision: 0.039749644087119534
train Recall: 0.02622244343669577


Epoch: 0: 100%|██████████| 3/3 [00:14<00:00,  4.98s/batch, loss=2.45, precision=0.0467, recall=0.0267]


val Loss: 2.5681925614674888
val Precision: 0.04708056027690569
val Recall: 0.019407511688768864


...

val Loss: 1.3173669576644897
val Precision: 0.2124592661857605
val Recall: 0.24541036784648895


Epoch: 49: 100%|██████████| 220/220 [04:48<00:00,  1.31s/batch, loss=1.33, precision=0.234, recall=0.208] 


train Loss: 1.1675486312671142
train Precision: 0.2649217103015293
train Recall: 0.2738180441612547


Epoch: 49: 100%|██████████| 3/3 [00:15<00:00,  5.03s/batch, loss=1.23, precision=0.173, recall=0.192]


val Loss: 1.2641300757726033
val Precision: 0.22487195829550424
val Recall: 0.2438757469256719
Weight saved for epoch 49.


In [14]:
batch_size = 228
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_v2/', epochs=50, load_weights='../../weights/torch_with_momentum/49.pth')

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0


train_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_precision,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████
train_recall,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████
val_loss,█▇▇▇▆▆▅▅▅▅▅▅▄▄▄▅▄▄▃▄▃▃▃▂▂▂▂▃▃▃▂▂▃▁▂▁▁▁▁▁
val_precision,▁▁▂▂▂▃▃▃▄▄▅▄▅▅▄▄▅▅▆▅▅▆▇▇▆▇▇▆▆▆▇▆▆▇▇▇▇███
val_recall,▁▁▁▂▂▃▃▃▄▄▅▄▅▄▅▄▄▅▆▅▅▆▆▇▆▇▇▆▇▆▇▇▆█▇▇▇▇▇█
train_loss,1.16755
train_precision,0.26492
train_recall,0.27382
val_loss,1.26413
val_precision,0.22487


Epoch: 0: 100%|██████████| 220/220 [04:48<00:00,  1.31s/batch, loss=1.11, precision=0.245, recall=0.268] 


train Loss: 1.1537194289944388
train Precision: 0.2702697280455719
train Recall: 0.2793115023862232


Epoch: 0: 100%|██████████| 3/3 [00:15<00:00,  5.02s/batch, loss=1.32, precision=0.164, recall=0.164]


...

val Loss: 0.9546052614847819
val Precision: 0.2976718842983246
val Recall: 0.29288868109385174


Epoch: 49: 100%|██████████| 220/220 [04:47<00:00,  1.31s/batch, loss=0.97, precision=0.37, recall=0.395]  


train Loss: 0.8159813989292491
train Precision: 0.3623996901241216
train Recall: 0.37760404321280394


Epoch: 49: 100%|██████████| 3/3 [00:15<00:00,  5.07s/batch, loss=0.933, precision=0.243, recall=0.22] 


val Loss: 0.9064188400904337
val Precision: 0.2875457356373469
val Recall: 0.30387507875760394
Weight saved for epoch 49.


In [14]:
batch_size = 256
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_v3/', epochs=250)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 196/196 [04:22<00:00,  1.34s/batch, loss=2.8, precision=0.0379, recall=0.0166] 


train Loss: 2.794513634272984
train Precision: 0.041099494988364835
train Recall: 0.02457012413829869


Epoch: 0: 100%|██████████| 2/2 [00:17<00:00,  8.62s/batch, loss=2.56, precision=0.0496, recall=0.0252]


val Loss: 2.608085036277771
val Precision: 0.05098747834563255
val Recall: 0.02587014250457287
Weight saved for epoch 0.


...

train Loss: 0.4073923087241698
train Precision: 0.5198577829465574
train Recall: 0.5331252992761378


Epoch: 196: 100%|██████████| 2/2 [00:17<00:00,  8.98s/batch, loss=0.566, precision=0.471, recall=0.513]


val Loss: 0.6619004905223846
val Precision: 0.4407465308904648
val Recall: 0.4934449940919876
Weight saved for epoch 196.


Epoch: 197:  50%|█████     | 98/196 [02:36<02:36,  1.60s/batch, loss=0.375, precision=0.506, recall=0.505] 


KeyboardInterrupt: 